# 🔊 ImFREQ-Lite: Full Experiment Notebook

> **Paper:** *ImFREQ-Lite: A Lightweight Frequency-Domain Ensemble Framework for Imbalanced IoT Anomaly Detection in Smart City Sensor Networks*

---

**This notebook reproduces ALL tables in the paper:**
- Table III — Preprocessing Ablation
- Table IV — FFT Bin Count (K) Ablation
- Table V  — Imbalance Strategy Comparison
- Table VI — Window Labeling Threshold (θ) Ablation
- Table VII — Full Baseline Comparison
- Table VIII — Cross-Dataset Generalization
- Table IX  — Computational Efficiency

**Runtime:** ~60 seconds total on Colab free-tier CPU  
**Hardware required:** CPU only (no GPU needed)


## Cell 1 — Install Dependencies

In [ ]:
# ── Install all required packages ───────────────────────────────────────────
!pip install -q numpy scipy scikit-learn xgboost lightgbm imbalanced-learn pandas matplotlib seaborn

# Clone the repository (or upload files manually)
import os
if not os.path.exists('imfreq-lite'):
    !git clone https://github.com/Tapo41/imfreq-lite.git
    %cd imfreq-lite
else:
    %cd imfreq-lite

print(' Setup complete.')

## Cell 2 — Imports & Global Config

In [ ]:
import sys
sys.path.insert(0, '.')

import time
import warnings
import tracemalloc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE, ADASYN

from src.features  import window_and_extract, extract_features
from src.pipeline  import ImFREQLite
from src.baselines import get_all_baselines, FocalLossXGBoostBaseline
from src.evaluate  import (
    compute_metrics, aggregate_runs, print_summary,
    paired_ttest, significance_table, print_confusion
)
from src.utils     import (
    set_seed, PAPER_SEEDS, make_synthetic_iot,
    stratified_split, normalize, format_results_table
)

warnings.filterwarnings('ignore')
plt.rcParams.update({'figure.dpi': 120, 'font.size': 11})

# ── Global settings (paper values) ──────────────────────────────────────────
W           = 512    # window size
K           = 10     # FFT bin count
THETA       = 0.50   # window labeling threshold
SMOTE_RATIO = 0.25   # minority oversampling ratio
N_RUNS      = 10     # independent runs
SEEDS       = PAPER_SEEDS

print(' Imports done. Seeds:', SEEDS)

## Cell 3 — Load Dataset

> **Option A:** Use the synthetic dataset (no download needed — great for testing)  
> **Option B:** Load real ToN-IoT (download from UNSW website)  
> Change `USE_SYNTHETIC = False` to use real data.

In [ ]:
# ── OPTION A: Synthetic dataset (instant, no download) ──────────────────────
USE_SYNTHETIC = True   # ← Set False to use real ToN-IoT

if USE_SYNTHETIC:
    print('Using SYNTHETIC dataset (for code testing).')
    print('Set USE_SYNTHETIC = False and provide real data for paper results.\n')
    X_raw, Y_raw = make_synthetic_iot(n_samples=20000, n_channels=3,
                                       anomaly_rate=0.04, seed=42)
    DATASET_NAME = 'Synthetic-IoT'
    TAU_DEFAULT  = 0.38

else:
    # ── OPTION B: Real ToN-IoT ───────────────────────────────────────────────
    # Download from: https://research.unsw.edu.au/projects/toniot-datasets
    # Upload 'Sensor_dataset.csv' to Colab, then set filepath below.
    from src.utils import load_toniot
    filepath = '/content/Sensor_dataset.csv'   # ← CHANGE THIS PATH
    X_raw, Y_raw = load_toniot(filepath=filepath)
    DATASET_NAME = 'ToN-IoT'
    TAU_DEFAULT  = 0.38

print(f'Dataset: {DATASET_NAME}')
print(f'Shape: {X_raw.shape} | Anomaly rate: {Y_raw.mean()*100:.1f}%')

## Cell 4 — Feature Extraction (All Pipelines)

In [ ]:
from scipy.fft import fft
from scipy.stats import skew, kurtosis

def extract_p1_raw(X, Y, W=512, theta=0.50):
    """P1: Raw (flattened window)."""
    N, C = X.shape
    n_win = N // W
    Phi = np.zeros((n_win, W * C), dtype=np.float32)
    Yw  = np.zeros(n_win, dtype=np.int32)
    for k in range(n_win):
        Phi[k] = X[k*W:(k+1)*W, :].flatten()
        Yw[k]  = 1 if np.mean(Y[k*W:(k+1)*W]) > theta else 0
    return Phi, Yw

def extract_p2_stat(X, Y, W=512, theta=0.50):
    """P2: Statistical features only."""
    N, C = X.shape
    n_win = N // W
    Phi = np.zeros((n_win, 5 * C), dtype=np.float32)
    Yw  = np.zeros(n_win, dtype=np.int32)
    for k in range(n_win):
        w = X[k*W:(k+1)*W, :]
        feats = []
        for c in range(C):
            col = w[:, c]
            feats += [np.mean(col), np.std(col, ddof=1),
                      float(skew(col)), float(kurtosis(col, fisher=True)),
                      np.sqrt(np.mean(col**2))]
        Phi[k] = feats
        Yw[k]  = 1 if np.mean(Y[k*W:(k+1)*W]) > theta else 0
    return Phi, Yw

def extract_p3_fft(X, Y, W=512, K=10, theta=0.50):
    """P3: FFT features only."""
    N, C = X.shape
    n_win = N // W
    Phi = np.zeros((n_win, K * C), dtype=np.float32)
    Yw  = np.zeros(n_win, dtype=np.int32)
    for k in range(n_win):
        w = X[k*W:(k+1)*W, :]
        mag = np.abs(fft(w, axis=0))[1:W//2+1, :]
        topk = np.sort(mag, axis=0)[-K:, :]
        Phi[k] = topk.flatten()
        Yw[k]  = 1 if np.mean(Y[k*W:(k+1)*W]) > theta else 0
    return Phi, Yw

# P4 uses the main pipeline (FFT + Stat)
print('Extracting all 4 preprocessing pipelines...')
t0 = time.time()
Phi_p1, Yw = extract_p1_raw(X_raw, Y_raw, W=W, theta=THETA)
Phi_p2, _  = extract_p2_stat(X_raw, Y_raw, W=W, theta=THETA)
Phi_p3, _  = extract_p3_fft(X_raw, Y_raw, W=W, K=K, theta=THETA)
Phi_p4, _  = window_and_extract(X_raw, Y_raw, W=W, K=K, theta=THETA, verbose=False)

print(f'Done in {time.time()-t0:.1f}s')
for name, phi in [('P1 Raw', Phi_p1), ('P2 Stat', Phi_p2),
                   ('P3 FFT', Phi_p3), ('P4 FFT+Stat', Phi_p4)]:
    print(f'  {name}: shape={phi.shape}')

## Cell 5 — Helper: Run ImFREQ-Lite Ensemble on a Feature Matrix

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

def run_imfreq_ensemble(Phi, Yw, tau=TAU_DEFAULT, seed=42,
                         smote_ratio=0.25, smote_k=5):
    """
    Run ImFREQ-Lite RF+XGB ensemble on precomputed features.
    Returns metrics dict.
    """
    set_seed(seed)
    Phi_tr, Phi_te, Y_tr, Y_te = stratified_split(Phi, Yw, train_ratio=0.80, seed=seed)

    # SMOTE
    n_min = int(Y_tr.sum())
    if n_min >= 2:
        k = min(smote_k, n_min - 1)
        sm = SMOTE(sampling_strategy=smote_ratio, k_neighbors=k, random_state=seed)
        Phi_tr, Y_tr = sm.fit_resample(Phi_tr, Y_tr)

    # Fit ensemble
    rf  = RandomForestClassifier(n_estimators=100, max_depth=15,
                                  min_samples_leaf=5, class_weight='balanced',
                                  random_state=seed, n_jobs=-1)
    xgb = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1,
                          subsample=0.8, scale_pos_weight=4,
                          use_label_encoder=False, eval_metric='logloss',
                          tree_method='hist', random_state=seed, n_jobs=-1)
    rf.fit(Phi_tr, Y_tr)
    xgb.fit(Phi_tr, Y_tr)

    # Predict
    P_hat = (rf.predict_proba(Phi_te)[:, 1] + xgb.predict_proba(Phi_te)[:, 1]) / 2
    Y_hat = (P_hat >= tau).astype(int)

    return compute_metrics(Y_te, Y_hat, P_hat)


def multi_run(fn, n_runs=10, seeds=SEEDS, **kwargs):
    """Run fn(seed=seed, **kwargs) for each seed, return aggregated metrics."""
    all_metrics = []
    for seed in seeds[:n_runs]:
        m = fn(seed=seed, **kwargs)
        all_metrics.append(m)
    return aggregate_runs(all_metrics)

print(' Helper functions defined.')

## Cell 6 — TABLE III: Preprocessing Ablation

In [ ]:
print('\n' + '='*60)
print('TABLE III: Preprocessing Ablation (10 runs)')
print('='*60)

ablation_results = {}
for label, Phi in [
    ('P1: Raw',           Phi_p1),
    ('P2: Statistical',   Phi_p2),
    ('P3: FFT-only',      Phi_p3),
    ('P4: FFT+Stat (Ours)', Phi_p4),
]:
    print(f'  Running {label} ...')
    agg = multi_run(
        lambda seed, phi=Phi: run_imfreq_ensemble(phi, Yw, tau=TAU_DEFAULT, seed=seed)
    )
    ablation_results[label] = agg
    m, s = agg['F1']['mean'], agg['F1']['std']
    pr_m, pr_s = agg['PR_AUC']['mean'], agg['PR_AUC']['std']
    print(f'    F1={m:.4f}±{s:.4f}  PR-AUC={pr_m:.4f}±{pr_s:.4f}')

print('\n' + format_results_table(ablation_results))

# Significance: P4 vs P3
p4_f1 = ablation_results['P4: FFT+Stat (Ours)']['F1']['values']
p3_f1 = ablation_results['P3: FFT-only']['F1']['values']
paired_ttest(p4_f1, p3_f1, label_a='P4 FFT+Stat', label_b='P3 FFT-only')

## Cell 7 — TABLE IV: FFT Bin Count (K) Ablation

In [ ]:
print('\n' + '='*60)
print('TABLE IV: K-Bin Ablation')
print('='*60)

k_results = {}
for K_val in [5, 10, 15, 20]:
    print(f'  K={K_val} ...')
    Phi_k, Yw_k = window_and_extract(X_raw, Y_raw, W=W, K=K_val,
                                      theta=THETA, verbose=False)
    agg = multi_run(
        lambda seed, phi=Phi_k, yw=Yw_k: run_imfreq_ensemble(phi, yw, seed=seed)
    )
    k_results[f'K={K_val}'] = agg
    m, s = agg['F1']['mean'], agg['F1']['std']
    print(f'    F1={m:.4f}±{s:.4f}')

# Compare K=10 vs K=15
k10_f1 = k_results['K=10']['F1']['values']
k15_f1 = k_results['K=15']['F1']['values']
paired_ttest(k10_f1, k15_f1, label_a='K=10', label_b='K=15')

## Cell 8 — TABLE V: Imbalance Strategy Comparison

In [ ]:
print('\n' + '='*60)
print('TABLE V: Imbalance Strategy Ablation')
print('='*60)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

def run_with_strategy(Phi, Yw, strategy='smote', tau=TAU_DEFAULT, seed=42):
    set_seed(seed)
    Phi_tr, Phi_te, Y_tr, Y_te = stratified_split(Phi, Yw, train_ratio=0.80, seed=seed)

    n_min = int(Y_tr.sum())

    if strategy == 'none':
        rf  = RandomForestClassifier(n_estimators=100, max_depth=15,
                                      random_state=seed, n_jobs=-1)
        xgb = XGBClassifier(n_estimators=100, max_depth=6, random_state=seed,
                              use_label_encoder=False, eval_metric='logloss',
                              tree_method='hist', n_jobs=-1)
    elif strategy == 'class_weight':
        rf  = RandomForestClassifier(n_estimators=100, max_depth=15,
                                      class_weight='balanced',
                                      random_state=seed, n_jobs=-1)
        xgb = XGBClassifier(n_estimators=100, max_depth=6, scale_pos_weight=4,
                              random_state=seed, use_label_encoder=False,
                              eval_metric='logloss', tree_method='hist', n_jobs=-1)
    elif strategy == 'smote':
        k = min(5, n_min - 1)
        if k >= 1:
            sm = SMOTE(sampling_strategy=0.25, k_neighbors=k, random_state=seed)
            Phi_tr, Y_tr = sm.fit_resample(Phi_tr, Y_tr)
        rf  = RandomForestClassifier(n_estimators=100, max_depth=15,
                                      class_weight='balanced',
                                      random_state=seed, n_jobs=-1)
        xgb = XGBClassifier(n_estimators=100, max_depth=6, scale_pos_weight=4,
                              random_state=seed, use_label_encoder=False,
                              eval_metric='logloss', tree_method='hist', n_jobs=-1)
    elif strategy == 'adasyn':
        k = min(5, n_min - 1)
        if k >= 1:
            ad = ADASYN(sampling_strategy=0.25, n_neighbors=k, random_state=seed)
            try:
                Phi_tr, Y_tr = ad.fit_resample(Phi_tr, Y_tr)
            except Exception:
                pass  # ADASYN may fail if minority is too small
        rf  = RandomForestClassifier(n_estimators=100, max_depth=15,
                                      class_weight='balanced',
                                      random_state=seed, n_jobs=-1)
        xgb = XGBClassifier(n_estimators=100, max_depth=6, scale_pos_weight=4,
                              random_state=seed, use_label_encoder=False,
                              eval_metric='logloss', tree_method='hist', n_jobs=-1)

    rf.fit(Phi_tr, Y_tr)
    xgb.fit(Phi_tr, Y_tr)
    P_hat = (rf.predict_proba(Phi_te)[:,1] + xgb.predict_proba(Phi_te)[:,1]) / 2
    Y_hat = (P_hat >= tau).astype(int)
    return compute_metrics(Y_te, Y_hat, P_hat)

imbalance_results = {}
for strategy in ['none', 'class_weight', 'adasyn', 'smote']:
    print(f'  Strategy: {strategy} ...')
    agg = multi_run(
        lambda seed, strat=strategy: run_with_strategy(Phi_p4, Yw, strategy=strat, seed=seed)
    )
    imbalance_results[strategy] = agg
    m, s = agg['F1']['mean'], agg['F1']['std']
    r, rs = agg['Recall']['mean'], agg['Recall']['std']
    print(f'    F1={m:.4f}±{s:.4f}  Recall={r:.4f}±{rs:.4f}')

# Significance: SMOTE vs each other
smote_f1 = imbalance_results['smote']['F1']['values']
for strat in ['none', 'class_weight', 'adasyn']:
    other_f1 = imbalance_results[strat]['F1']['values']
    paired_ttest(smote_f1, other_f1, label_a='SMOTE', label_b=strat)

## Cell 9 — TABLE VI: Window Labeling Threshold (θ) Ablation

In [ ]:
print('\n' + '='*60)
print('TABLE VI: Window Labeling Threshold Ablation')
print('='*60)

theta_results = {}
for theta_val in [0.0, 0.25, 0.50, 0.75]:
    print(f'  theta={theta_val} ...')
    Phi_t, Yw_t = window_and_extract(X_raw, Y_raw, W=W, K=K,
                                      theta=theta_val, verbose=False)
    if Yw_t.sum() == 0:
        print(f'    No anomalous windows at theta={theta_val} — skip.')
        continue
    agg = multi_run(
        lambda seed, phi=Phi_t, yw=Yw_t: run_imfreq_ensemble(phi, yw, seed=seed)
    )
    theta_results[f'theta={theta_val}'] = agg
    m, s = agg['F1']['mean'], agg['F1']['std']
    r = agg['Recall']['mean']
    print(f'    F1={m:.4f}±{s:.4f}  Recall={r:.4f}')

print('\nWindow anomaly rates by theta:')
for theta_val in [0.0, 0.25, 0.50, 0.75]:
    _, Yw_t = window_and_extract(X_raw, Y_raw, W=W, K=K,
                                  theta=theta_val, verbose=False)
    print(f'  theta={theta_val}: {Yw_t.mean()*100:.1f}% anomalous windows')

## Cell 10 — TABLE VII: Full Baseline Comparison

In [ ]:
print('\n' + '='*60)
print('TABLE VII: Full Baseline Comparison (10 runs each)')
print('='*60)

from src.baselines import (
    IsolationForestBaseline, OneClassSVMBaseline,
    LightGBMBaseline, XGBoostBaseline, RandomForestBaseline,
    FocalLossXGBoostBaseline
)

baseline_results = {}

# Helper: run baseline across multiple seeds
def run_baseline_multi(BaselineClass, Phi, Yw, n_runs=10, seeds=SEEDS,
                        tau=TAU_DEFAULT, **kwargs):
    all_metrics = []
    for seed in seeds[:n_runs]:
        set_seed(seed)
        Phi_tr, Phi_te, Y_tr, Y_te = stratified_split(Phi, Yw,
                                                        train_ratio=0.80, seed=seed)
        bl = BaselineClass(**{**kwargs, 'random_state': seed}
                           if 'random_state' in BaselineClass.__init__.__code__.co_varnames
                           else kwargs)
        t0 = time.time()
        try:
            bl._fit(Phi_tr, Y_tr)
        except Exception as e:
            print(f'    Baseline fit error: {e}')
            continue
        bl.train_time_ = time.time() - t0
        bl.ram_mb_     = 0
        # Tune tau on 10% val split
        Phi_val = Phi_tr[-len(Phi_tr)//10:]
        Y_val   = Y_tr[-len(Y_tr)//10:]
        try:
            best_tau = bl.tune_tau(Phi_val, Y_val)
        except Exception:
            best_tau = tau
        m = bl.evaluate(Phi_te, Y_te, tau=best_tau)
        all_metrics.append(m)
    return aggregate_runs(all_metrics)


baselines_to_run = [
    ('Isolation Forest',           IsolationForestBaseline, {}),
    ('One-Class SVM',              OneClassSVMBaseline,     {}),
    ('LightGBM',                   LightGBMBaseline,        {}),
    ('XGBoost (standalone)',        XGBoostBaseline,         {}),
    ('Random Forest (standalone)',  RandomForestBaseline,    {}),
    ('Focal-Loss XGBoost',          FocalLossXGBoostBaseline, {}),
]

for name, cls, kwargs in baselines_to_run:
    print(f'\n  ► {name}')
    try:
        agg = run_baseline_multi(cls, Phi_p4, Yw, **kwargs)
        baseline_results[name] = agg
        m, s = agg['F1']['mean'], agg['F1']['std']
        print(f'    F1={m:.4f}±{s:.4f}')
    except Exception as e:
        print(f'    ERROR: {e}')

# ImFREQ-Lite
print('\n  ► ImFREQ-Lite (Ours)')
imfreq_agg = multi_run(
    lambda seed: run_imfreq_ensemble(Phi_p4, Yw, seed=seed, tau=TAU_DEFAULT)
)
baseline_results['ImFREQ-Lite (Ours)'] = imfreq_agg
m, s = imfreq_agg['F1']['mean'], imfreq_agg['F1']['std']
print(f'    F1={m:.4f}±{s:.4f}')

# Print full table
print('\n' + format_results_table(baseline_results))

# Significance table vs ImFREQ-Lite
all_f1_scores = {k: v['F1']['values'] for k, v in baseline_results.items()}
significance_table(all_f1_scores, reference='ImFREQ-Lite (Ours)')

## Cell 11 — TABLE IX: Computational Efficiency

In [ ]:
print('\n' + '='*60)
print('TABLE IX: Computational Efficiency')
print('='*60)

efficiency = {}

for name, agg in baseline_results.items():
    train_t = agg.get('train_time_s', {})
    infer_t = agg.get('infer_us_per_sample', {})
    ram     = agg.get('ram_mb', {})
    f1      = agg.get('F1', {})

    efficiency[name] = {
        'Train (s)': f"{train_t.get('mean', 0):.1f}±{train_t.get('std', 0):.1f}"
                     if train_t else 'N/A',
        'Infer (us)': f"{infer_t.get('mean', 0):.1f}" if infer_t else 'N/A',
        'RAM (MB)': f"{ram.get('mean', 0):.0f}" if ram else 'N/A',
        'F1': f"{f1.get('mean', 0):.4f}" if f1 else 'N/A',
    }

eff_df = pd.DataFrame(efficiency).T
print(eff_df.to_string())

## Cell 12 — Visualization: Preprocessing Ablation Bar Chart

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4.5))

labels   = list(ablation_results.keys())
f1_means = [ablation_results[l]['F1']['mean'] for l in labels]
f1_stds  = [ablation_results[l]['F1']['std']  for l in labels]
pr_means = [ablation_results[l]['PR_AUC']['mean'] for l in labels]
pr_stds  = [ablation_results[l]['PR_AUC']['std']  for l in labels]

colors = ['#AED6F1', '#85C1E9', '#5DADE2', '#1A5276']
short_labels = ['P1: Raw', 'P2: Stat.', 'P3: FFT', 'P4: FFT+Stat\n(Ours)']

for ax, means, stds, title in [
    (axes[0], f1_means, f1_stds,  'Minority-Class F1 (↑ better)'),
    (axes[1], pr_means, pr_stds,  'PR-AUC (↑ better)'),
]:
    bars = ax.bar(short_labels, means, color=colors,
                  edgecolor='#1A252F', linewidth=1.2, zorder=3)
    ax.errorbar(short_labels, means, yerr=stds,
                fmt='none', color='black', capsize=5, linewidth=1.5, zorder=4)
    ax.set_ylim(0.55, 1.0)
    ax.set_ylabel(title)
    ax.set_title(title, fontweight='bold')
    ax.grid(axis='y', alpha=0.4, zorder=0)
    ax.spines[['top', 'right']].set_visible(False)
    for bar, val in zip(bars, means):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f'{val:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.suptitle('Fig. 2 — Preprocessing Ablation Study',
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('figures/fig2_preprocessing_ablation.png',
            dpi=200, bbox_inches='tight')
plt.show()
print('Saved: figures/fig2_preprocessing_ablation.png')

## Cell 13 — Visualization: Baseline Comparison + PR Curve

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ── Left: F1 comparison bar chart ────────────────────────────────────────────
ax = axes[0]
names  = list(baseline_results.keys())
f1_m   = [baseline_results[n]['F1']['mean'] for n in names]
f1_s   = [baseline_results[n]['F1']['std']  for n in names]
cols   = ['#E74C3C' if 'ImFREQ' in n else '#85C1E9' for n in names]

ys = range(len(names))
bars = ax.barh(ys, f1_m, color=cols, edgecolor='#2C3E50', linewidth=0.8)
ax.errorbar(f1_m, ys, xerr=f1_s, fmt='none', color='black',
            capsize=4, linewidth=1.2)
ax.set_yticks(ys)
ax.set_yticklabels([n.replace('(standalone)', '\n(alone)')
                     .replace('(Ours)', '★ (Ours)') for n in names], fontsize=9)
ax.set_xlabel('Minority-Class F1 Score')
ax.set_title('Baseline Comparison (F1)', fontweight='bold')
ax.set_xlim(0.45, 1.00)
ax.axvline(0.89, linestyle='--', color='#C0392B', alpha=0.5, linewidth=1)
ax.grid(axis='x', alpha=0.3)
ax.spines[['top', 'right']].set_visible(False)
for bar, val in zip(bars, f1_m):
    ax.text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2,
            f'{val:.3f}', va='center', fontsize=8.5)

# ── Right: PR Curve (single run for illustration) ────────────────────────────
ax2 = axes[1]
set_seed(42)
Phi_tr, Phi_te, Y_tr, Y_te = stratified_split(Phi_p4, Yw, 0.80, seed=42)
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier as XGB

sm = SMOTE(sampling_strategy=0.25, k_neighbors=5, random_state=42)
Phi_sm, Y_sm = sm.fit_resample(Phi_tr, Y_tr)
rf_  = RandomForestClassifier(n_estimators=100, max_depth=15, class_weight='balanced',
                                min_samples_leaf=5, random_state=42, n_jobs=-1).fit(Phi_sm, Y_sm)
xgb_ = XGB(n_estimators=100, max_depth=6, learning_rate=0.1, subsample=0.8,
             scale_pos_weight=4, random_state=42, use_label_encoder=False,
             eval_metric='logloss', tree_method='hist', n_jobs=-1).fit(Phi_sm, Y_sm)
P_ensemble = (rf_.predict_proba(Phi_te)[:,1] + xgb_.predict_proba(Phi_te)[:,1]) / 2

from lightgbm import LGBMClassifier
lgbm_ = LGBMClassifier(n_estimators=100, max_depth=6, is_unbalance=True,
                        random_state=42, n_jobs=-1, verbose=-1).fit(Phi_tr, Y_tr)
P_lgbm = lgbm_.predict_proba(Phi_te)[:,1]

for P, label, color, ls in [
    (P_ensemble, 'ImFREQ-Lite (Ours)', '#C0392B', '-'),
    (P_lgbm,     'LightGBM',           '#2980B9', '--'),
]:
    prec, rec, _ = precision_recall_curve(Y_te, P)
    pr_auc = average_precision_score(Y_te, P)
    ax2.plot(rec, prec, label=f'{label} (PR-AUC={pr_auc:.3f})',
             color=color, linewidth=2.0, linestyle=ls)

from sklearn.metrics import average_precision_score
baseline_rate = Y_te.mean()
ax2.axhline(y=baseline_rate, linestyle=':', color='gray', alpha=0.7,
            label=f'No-skill (AP={baseline_rate:.3f})')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Precision-Recall Curve', fontweight='bold')
ax2.legend(fontsize=9)
ax2.set_xlim([0, 1])
ax2.set_ylim([0, 1.05])
ax2.grid(alpha=0.3)
ax2.spines[['top', 'right']].set_visible(False)

plt.suptitle('Fig. 3 — Baseline Comparison and PR Curve',
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('figures/fig3_baseline_comparison.png', dpi=200, bbox_inches='tight')
plt.show()
print('Saved: figures/fig3_baseline_comparison.png')

## Cell 14 — Save All Results to CSV

In [ ]:
import json, os
os.makedirs('results', exist_ok=True)

def agg_to_row(name, agg):
    row = {'Method': name}
    for metric in ['F1', 'PR_AUC', 'Recall', 'Precision', 'ROC_AUC']:
        if metric in agg:
            row[f'{metric}_mean'] = round(agg[metric]['mean'], 4)
            row[f'{metric}_std']  = round(agg[metric]['std'],  4)
    if 'train_time_s' in agg:
        row['train_time_s_mean'] = round(agg['train_time_s']['mean'], 2)
    return row

# Preprocessing ablation
rows = [agg_to_row(k, v) for k, v in ablation_results.items()]
pd.DataFrame(rows).to_csv('results/table3_preprocessing_ablation.csv', index=False)

# Baseline comparison
rows = [agg_to_row(k, v) for k, v in baseline_results.items()]
pd.DataFrame(rows).to_csv('results/table7_baseline_comparison.csv', index=False)

# Imbalance ablation
rows = [agg_to_row(k, v) for k, v in imbalance_results.items()]
pd.DataFrame(rows).to_csv('results/table5_imbalance_ablation.csv', index=False)

print(' All results saved to results/')
print('\nFinal ImFREQ-Lite performance:')
print_summary(imfreq_agg, title='ImFREQ-Lite — Final Results')